# Data Cleaning for Analysis on Movies

![Data Cleaning](images/theater_cleaning.jpg "Data Cleaning")

_Grab the cleaning tools and Let's start with the cleaning._

### Import required libraries

In [1]:
# for data cleaning
import pandas as pd
import numpy as np

# for loading all datasets in one go
from glob import glob

# for ignoring some future warnings
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

### Loading all the datasets

In [2]:
# get all files
files = glob('datasets/*.csv')

# create dict with keys as file name and value and file
datasets = { '_'.join(f.replace('datasets\\','').replace('.csv','').split('.')): pd.read_csv(f) for f in files }
# if using other linux or mac replace f.replace('datasets\\','') with f.replace('datasets/','')

# name of all the datasets
datasetsName = datasets.keys()
# ['imdb_title_principals', 'imdb_name_basics','imdb_title_basics', 'title_ratings', 'tn_movie_budgets']

# Data Cleaning

## 1.1 Cleaning The Numbers Movie Budgets DataFrame

 _Let's load the `tn_movie_budgets` datasets_

In [3]:
df_tn_movie_budgets = datasets['tn_movie_budgets']
df_tn_movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


_Check the number of rows and columns it has_

In [4]:
df_tn_movie_budgets.shape

(5782, 6)

_Check the datatype and counts by displaying dataframe info_

In [5]:
df_tn_movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


_Check for the missing values_

In [6]:
df_tn_movie_budgets.isna().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

_As we can see that we don't have any missing values so we can move forward with the next step_.

_The column `release_date` has object datatype lets make it year only and datatype as pandas year_

In [7]:
df_tn_movie_budgets['year'] = pd.to_datetime(df_tn_movie_budgets['release_date']).dt.year
df_tn_movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279",2009
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",2019
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",2017


_Check how many years of data we have_

In [8]:
print(f"Data From Year {df_tn_movie_budgets.year.min()} to {df_tn_movie_budgets.year.max()}")

Data From Year 1915 to 2020


_As we can see we have data from 1915 to 2020. Considering the inflation over the years old data might not be very useful so lets filter out data of last 15 years only i.e from 2006 to 2020_

In [9]:
df_tn_movie_budgets = df_tn_movie_budgets.loc[df_tn_movie_budgets['year'] >= 2006, : ]
df_tn_movie_budgets.shape

(3177, 7)

In [10]:
print(f"Data From Year {df_tn_movie_budgets.year.min()} to {df_tn_movie_budgets.year.max()}")

Data From Year 2006 to 2020


In [11]:
df_tn_movie_budgets.year.value_counts()

2015    338
2010    274
2008    264
2006    260
2014    255
2011    254
2009    239
2013    238
2012    235
2007    220
2016    219
2017    168
2018    143
2019     67
2020      3
Name: year, dtype: int64

_The columns `production_budget`, `domestic_gross` and `worldwide_gross` has $ and comma (,) in amount value which needs to be convrted to int value_

In [12]:
def convert_amt_to_int(df,col):
    df[col] = df[col].replace('[\$,]', '', regex=True).astype(np.int64)
    return df

In [13]:
amount_cols = ['production_budget', 'domestic_gross', 'worldwide_gross']

for col in amount_cols:
    df_tn_movie_budgets = convert_amt_to_int(df_tn_movie_budgets,col)
    

df_tn_movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3177 entries, 0 to 5780
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 3177 non-null   int64 
 1   release_date       3177 non-null   object
 2   movie              3177 non-null   object
 3   production_budget  3177 non-null   int64 
 4   domestic_gross     3177 non-null   int64 
 5   worldwide_gross    3177 non-null   int64 
 6   year               3177 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 198.6+ KB


_Let's calculate ROI as we have cost of production and worldwide collection_

In [14]:
df_tn_movie_budgets['ROI'] = df_tn_movie_budgets.apply(lambda x: ((x['worldwide_gross'] - x['production_budget']) / x['production_budget']) * 100 , axis=1)
df_tn_movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year,ROI
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2009,553.257713
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011,154.667286
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,2019,-57.210757
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,2015,324.384139
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017,315.369636


_Let's add foreign gross as well_

In [15]:
df_tn_movie_budgets['foreign'] = df_tn_movie_budgets.apply(lambda x: x['worldwide_gross'] - x['domestic_gross'],axis=1)

In [16]:
df_tn_movie_budgets = df_tn_movie_budgets[['id','release_date','movie','production_budget','domestic_gross','foreign','worldwide_gross','ROI','year']]

In [17]:
df_tn_movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,foreign,worldwide_gross,ROI,year
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2015837654,2776345279,553.257713,2009
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,804600000,1045663875,154.667286,2011
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,107000000,149762350,-57.210757,2019
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,944008095,1403013963,324.384139,2015
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,696540365,1316721747,315.369636,2017


_Lets drop `id` and `release_date` as we don't need it_

In [18]:
df_tn_movie_budgets.drop(['id','release_date'],axis=1,inplace=True)
df_tn_movie_budgets.head()

,movie,production_budget,domestic_gross,foreign,worldwide_gross,ROI,year
0,Avatar,425000000,760507625,2015837654,2776345279,553.257713,2009
1,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,804600000,1045663875,154.667286,2011
2,Dark Phoenix,350000000,42762350,107000000,149762350,-57.210757,2019
3,Avengers: Age of Ultron,330600000,459005868,944008095,1403013963,324.384139,2015
4,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,696540365,1316721747,315.369636,2017


## 1.2 Cleaning and Merging Title Rating and Imdb Title DataFrame

_Lets load the `title_ratings` dataset_

In [19]:
df_ratings = datasets['title_ratings']
df_ratings.head()

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


_Check the total records_

In [20]:
df_ratings.shape

(73856, 3)

_Check the datatype and counts by displaying dataframe info_

In [21]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


_Check for missing values_

In [22]:
df_ratings.isna().sum()

tconst           0
averagerating    0
numvotes         0
dtype: int64

_As we can see that we don't have any missing values so we can move forward with the next step_.

_Lets load the `imdb_title_basics` dataset_

In [23]:
df_imdb_title = datasets['imdb_title_basics']
df_imdb_title.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


_Check the total records_

In [24]:
df_imdb_title.shape

(146144, 6)

_Check the datatype and counts by displaying dataframe info_

In [25]:
df_imdb_title.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


_Check for missing values_

In [26]:
df_imdb_title.isna().sum()

tconst                 0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
dtype: int64

_We have null values in Original Title, Runtime and Genres_.

_Lets first merge this 2 dataframes on tconst_

In [27]:
df_imdb = df_imdb_title.merge(df_ratings, left_on='tconst',right_on='tconst',how='inner')
df_imdb.shape

(73856, 8)

In [28]:
df_imdb.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119


_Now that we have merged ratings and title datasets. lets find out the director of the film using imdb_name_basics and imdb_title_principals datasets_

In [29]:
df_title_detail = datasets['imdb_title_principals']
df_title_detail.head()

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN
2,tt0111414,3,nm3739909,producer,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN,NaN
4,tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"


In [30]:
df_title_detail.shape

(1028186, 6)

_Check unique values in category column_

In [31]:
df_title_detail['category'].unique()

array(['actor', 'director', 'producer', 'editor', 'actress', 'composer',
       'cinematographer', 'writer', 'self', 'production_designer',
       'archive_footage', 'archive_sound'], dtype=object)

_As we can see that for each title we have director, actor, producer associated with it. As we are only intrested in director lets filter out all the titles where category is director_

In [32]:
df_makers = df_title_detail[df_title_detail['category'].isin(['director','producer','writer','cinematographer']) ]
df_makers.head()

,tconst,ordering,nconst,category,job,characters
1,tt0111414,2,nm0398271,director,NaN,NaN
2,tt0111414,3,nm3739909,producer,producer,NaN
8,tt0323808,5,nm0362736,director,NaN,NaN
9,tt0323808,6,nm0811056,producer,producer,NaN
10,tt0323808,7,nm0914939,producer,producer,NaN


In [33]:
df_names = datasets['imdb_name_basics']
df_names.head()

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


In [34]:
df_names.shape

(606648, 6)

_As we have names of people involved in making of movie (cast, producers, director etc) we will  with their names_

In [35]:
df_makers_names = df_makers.merge(df_names, left_on='nconst',right_on='nconst',how='inner')
df_makers_names.head()

,tconst,ordering,nconst,category,job,characters,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,tt0111414,2,nm0398271,director,NaN,NaN,Frank Howson,1952.0,NaN,"actor,writer,producer","tt0104271,tt0094789,tt0102076,tt0111414"
1,tt5573596,5,nm0398271,director,NaN,NaN,Frank Howson,1952.0,NaN,"actor,writer,producer","tt0104271,tt0094789,tt0102076,tt0111414"
2,tt0111414,3,nm3739909,producer,producer,NaN,Barry Porter-Robinson,NaN,NaN,"producer,art_department","tt0290884,tt0101374,tt0111414,tt1566940"
3,tt0323808,5,nm0362736,director,NaN,NaN,Robin Hardy,1929.0,2016.0,"actor,writer,director","tt0070917,tt0097375,tt0091032,tt0323808"
4,tt0323808,6,nm0811056,producer,producer,NaN,Peter Snell,1938.0,NaN,"producer,soundtrack,executive","tt0070917,tt2231097,tt0093771,tt0078836"


_Lets drop unwanted columns from df_makers_names

In [36]:
df_makers_names.drop(['ordering','characters','job','birth_year','death_year','primary_profession','known_for_titles'], axis=1, inplace=True)

In [37]:
df_makers_names.head()

,tconst,nconst,category,primary_name
0,tt0111414,nm0398271,director,Frank Howson
1,tt5573596,nm0398271,director,Frank Howson
2,tt0111414,nm3739909,producer,Barry Porter-Robinson
3,tt0323808,nm0362736,director,Robin Hardy
4,tt0323808,nm0811056,producer,Peter Snell


In [38]:
df_makers_names.isna().sum()

tconst          0
nconst          0
category        0
primary_name    0
dtype: int64

_We will be using this dataset to recommend directors/producers/writer for a certain kind of films Microsoft Studios might wanna make. But as we have dropped death_year column we have no way to know if that person is even alive but for for the sake of more number of data to work with we will consider as all of the person present in the datasets are alive_

_Now we need to merge Movie titles with the makers names_

In [39]:
#['director','producer','writer','cinematographer']

df_director = df_makers_names.loc[(df_makers_names['category'] == 'director'),['primary_name','tconst']].rename(columns = {'primary_name':'director'})
df_producer = df_makers_names.loc[(df_makers_names['category'] == 'producer'),['primary_name','tconst']].rename(columns = {'primary_name':'producer'})
df_writer = df_makers_names.loc[(df_makers_names['category'] == 'writer'),['primary_name','tconst']].rename(columns = {'primary_name':'writer'})
df_cinematographer = df_makers_names.loc[(df_makers_names['category'] == 'cinematographer'),['primary_name','tconst']].rename(columns = {'primary_name':'cinematographer'})


display(df_director.shape)
display(df_producer.shape)
display(df_writer.shape)
display(df_cinematographer.shape)

(146393, 2)

(113724, 2)

(74357, 2)

(80091, 2)

In [40]:
#merge directors
df_merge_d = df_imdb.merge(df_director, left_on='tconst',right_on='tconst',how='inner')
df_merge_d.rename(columns = {'primary_name':'director'}, inplace = True)

df_merge_d.shape

(77452, 9)

In [41]:
#merge producer
df_merge_p = df_merge_d.merge(df_producer, left_on='tconst',right_on='tconst',how='inner')
df_merge_p.rename(columns = {'primary_name':'producer'}, inplace = True)

df_merge_p.shape

(81671, 10)

In [42]:
#merge cinematographer
df_merge_c = df_merge_p.merge(df_cinematographer, left_on='tconst',right_on='tconst',how='inner')
df_merge_c.rename(columns = {'primary_name':'cinematographer'}, inplace = True)

df_merge_c.shape

(42711, 11)

In [43]:
#merge writer
df_merge_w = df_merge_c.merge(df_writer, left_on='tconst',right_on='tconst',how='inner')
df_merge_w.rename(columns = {'primary_name':'writer'}, inplace = True)

df_merge_w.shape

(17742, 12)

In [44]:
# lets copy it into a new variable
df_imdb_with_names = df_merge_w.copy()
df_imdb_with_names.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,director,producer,cinematographer,writer
0,tt0146592,Pál Adrienn,Pál Adrienn,2010,136.0,Drama,6.8,451,Ágnes Kocsis,Ferenc Pusztai,Ádám Fillenz,Andrea Roberti
1,tt0176694,The Tragedy of Man,Az ember tragédiája,2011,160.0,"Animation,Drama,History",7.8,584,Marcell Jankovics,Károlyné Koncz,Zoltán Bacsó,Imre Madách
2,tt0176694,The Tragedy of Man,Az ember tragédiája,2011,160.0,"Animation,Drama,History",7.8,584,Marcell Jankovics,Borbála Mezei,Zoltán Bacsó,Imre Madách
3,tt0293069,Dark Blood,Dark Blood,2012,86.0,Thriller,6.6,1053,George Sluizer,JoAnne Sellar,Edward Lachman,Jim Barton
4,tt0326592,The Overnight,The Overnight,2010,88.0,NaN,7.5,24,Jed I. Goodman,Sebastian Attie,Roderick E. Stevens,Kacey Arnold


_Lets find out if we have any duplicate entries based on tconst and primary_title_

In [45]:
df_duplicated_tconst = df_imdb_with_names[df_imdb_with_names.duplicated(['tconst'])]
df_duplicated_tconst.shape

(7358, 12)

_We have 7358 duplicated rows which has same value of tconst_

In [46]:
df_duplicated_title = df_imdb_with_names[df_imdb_with_names.duplicated(['primary_title'])]
df_duplicated_title.shape

(7514, 12)

_We have 7514 duplicated rows which has same value of tconst_

_Now lets create a subset to drop all the dupicated rows and keep the first row_

In [47]:
df_imdb_with_names.drop_duplicates(subset=['tconst', 'primary_title'], keep='first',inplace=True)
df_imdb_with_names.shape

(10384, 12)

_Now its time to merge `df_imdb_with_names` with `df_tn_movie_budgets`_

_As `df_tn_movie_budgets` does not have tconst so we have to merge on title._

_Before moving forward lets get the SET of titles from each dataset_

In [48]:
tn_movie_budget_values = set(df_tn_movie_budgets['movie'].values) 
len(tn_movie_budget_values)

# there are total 3165 unique movie title in df_tn_movie_budget

3165

In [49]:
imdb_with_names_values = set(df_imdb_with_names['original_title'].values) 
len(imdb_with_names_values)

# there are total 10228 unique movie title in df_imdb_with_names

10281

In [50]:
# movie title present in tn_movie_budget_values but not in imdb_with_names_values

diff1 = tn_movie_budget_values - imdb_with_names_values
len(diff1)

2842

In [51]:
# movie title present in imdb_with_names_values but not in tn_movie_budget_values

diff2 = imdb_with_names_values - tn_movie_budget_values
len(diff2)

9958

_Diffrence Between total movies in tn_movie_budget_values and movies present in tn_movie_budget_values but not in imdb_with_names_values_

In [52]:
print(len(tn_movie_budget_values - diff1))

323


_Diffrence Between total movies in imdb_with_names_values and movies present in imdb_with_names_values but not in tn_movie_budget_values_

In [53]:
print(len(imdb_with_names_values - diff2))

323


_So we have 323 Movies with matching titles in both datasets. so when merged we must see 323 rows_

In [54]:
# Merge df_tn_movie_budgets into df_imdb_with_names

df = df_imdb_with_names.merge(df_tn_movie_budgets, left_on='original_title',right_on='movie',how='inner')
df.shape

(338, 19)

_Something has to be wrong we were expecting 323 but got 338 instead. Let's check if we have duplicated titles and if we do we need to drop those._

In [55]:
df_duplicate = df[df.duplicated(['original_title'])].shape
df_duplicate

(15, 19)

_So there are 15 duplicate rows and we must keep the first row and delete rest._

In [56]:
df.drop_duplicates(subset=['original_title'], keep='first',inplace=True)
df.shape

(323, 19)

_Looks good now. we got what we expected._

In the previous step all we did is match the string and merge else discard and in this process we lost a lot of data. 
There is something which might help us get something back from the part of data we lost.
Lets say for example we have 2 dataframes `df1` and `df2` both have `title` column and we are trying to merge if the title value matches. But what if `df1` has title as 'The fast and the furious' and `df2` has 'fast and the furious' as as both are same movie the only diffrence is that 'THE' is not present in `d2` and we can't afford to loose data just because of some missing characters.

In [57]:
#!pip install stringdist
import stringdist as sd

In [58]:
def leven(string1, string2):
    s1=[]
    s2=[]
    score=[]
    for i in string1:
        for j in string2:
            s1.append(i)
            s2.append(j)
            score.append(sd.levenshtein_norm(i, j))     
    ldf = pd.DataFrame({'s1':s1, 's2':s2, 'score':score})
    return ldf

In [59]:
string_match_score = leven(diff1,diff2)
#string_match_score = pd.read_csv('datasets/string_match_score.csv')
string_match_score.shape

(89, 4)

In [60]:
string_match_score = string_match_score[string_match_score['score']<0.20]
string_match_score.shape

(89, 4)

In [61]:
join1 = df_imdb_with_names.merge(string_match_score, left_on='original_title', right_on = 's2', how= 'inner')
join2 = join1.merge(df_tn_movie_budgets, left_on=['s1','start_year'], right_on = ['movie','year'], how='inner')

join2.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,director,producer,...,s1,s2,score,movie,production_budget,domestic_gross,foreign,worldwide_gross,ROI,year
0,tt0477302,Extremely Loud & Incredibly Close,Extremely Loud & Incredibly Close,2011,129.0,"Adventure,Drama,Mystery",6.9,93164,Stephen Daldry,Scott Rudin,...,Extremely Loud and Incredibly Close,Extremely Loud & Incredibly Close,0.085714,Extremely Loud and Incredibly Close,40000000,31847881,23400000,55247881,38.119703,2011
1,tt0901504,Psych:9,Psych:9,2010,98.0,"Horror,Mystery,Thriller",4.5,2072,Andrew Shortell,Philip Waley,...,Psych 9,Psych:9,0.142857,Psych 9,5000000,0,0,0,-100.000000,2010
2,tt1217613,Battle Los Angeles,Battle Los Angeles,2011,116.0,"Action,Sci-Fi",5.8,167052,Jonathan Liebesman,Neal H. Moritz,...,Battle: Los Angeles,Battle Los Angeles,0.052632,Battle: Los Angeles,70000000,83552429,129911547,213463976,204.948537,2011
3,tt1245492,This Is the End,This Is the End,2013,107.0,"Comedy,Fantasy",6.6,360556,Evan Goldberg,James Weaver,...,This is the End,This Is the End,0.066667,This is the End,32000000,101470202,25068915,126539117,295.434741,2013
4,tt1371150,This Is Where I Leave You,This Is Where I Leave You,2014,103.0,"Comedy,Drama",6.6,65202,Shawn Levy,Paula Weinstein,...,This is Where I Leave You,This Is Where I Leave You,0.040000,This is Where I Leave You,20000000,34296320,7000000,41296320,106.481600,2014


In [62]:
# df_matched_titles = pd.read_csv('datasets/matched_title_data.csv')

_We got 22 more movies to add it to our `df` dataframe So lets go and append it._

In [63]:
df = df.append(join2)
df.shape

(344, 23)

_Time to delete unwanted columns and rename_

In [64]:
df.drop(['tconst','primary_title','start_year','numvotes','s1','s2','score','movie','Unnamed: 0'], axis=1, inplace=True)

In [65]:
df.rename(columns = {'original_title':'title','runtime_minutes':'runtime','averagerating':'rating','production_budget':'budget','domestic_gross':'domestic','worldwide_gross':'worldwide'},inplace=True)

In [66]:
df.reset_index(drop=True,inplace=True)

In [67]:
df.head()

,title,runtime,genres,rating,director,producer,cinematographer,writer,budget,domestic,foreign,worldwide,ROI,year
0,The Overnight,88.0,NaN,7.5,Jed I. Goodman,Sebastian Attie,Roderick E. Stevens,Kacey Arnold,200000,1109808,56188,1165996,482.998000,2015
1,Lincoln,150.0,"Biography,Drama,History",7.4,Steven Spielberg,Kathleen Kennedy,Janusz Kaminski,Tony Kushner,65000000,182207973,91138308,273346281,320.532740,2012
2,Season of the Witch,95.0,"Action,Adventure,Fantasy",5.4,Dominic Sena,Charles Roven,Amir Mokri,Bragi F. Schut,40000000,24827228,66299372,91126600,127.816500,2011
3,Dallas Buyers Club,117.0,"Biography,Drama",8.0,Jean-Marc Vallée,Rachel Winter,Yves Bélanger,Craig Borten,5000000,27298285,33313560,60611845,1112.236900,2013
4,Interstellar,169.0,"Adventure,Drama,Sci-Fi",8.6,Christopher Nolan,Emma Thomas,Hoyte Van Hoytema,Jonathan Nolan,165000000,188017894,478361481,666379375,303.866288,2014


## 1.3 Handling Genre

_Lets get all the unique genre from the above dataframe_

In [68]:
df.dropna(subset=['genres'],inplace=True)

In [69]:
list_of_genres = df.genres.unique()
list_of_genres

array(['Biography,Drama,History', 'Action,Adventure,Fantasy',
       'Biography,Drama', 'Adventure,Drama,Sci-Fi',
       'Comedy,Drama,Romance', 'Action,Adventure,Sci-Fi', 'Drama,Romance',
       'Comedy', 'Action,Mystery,Thriller', 'Crime,Drama,Horror',
       'Action,Comedy,Crime', 'Action,Adventure,Drama',
       'Action,Fantasy,Horror', 'Action,Comedy,Romance',
       'Action,Family,Fantasy', 'Action,Crime,Sci-Fi',
       'Biography,Drama,Fantasy', 'Horror,Thriller',
       'Adventure,Comedy,Family', 'Horror', 'Action,Biography,Drama',
       'Biography,Comedy,Drama', 'Biography,Crime,Drama',
       'Drama,Mystery,Thriller', 'Animation,Comedy,Family',
       'Drama,Fantasy,Romance', 'Drama', 'Documentary',
       'Crime,Drama,Thriller', 'Adventure,Comedy,Fantasy',
       'Horror,Mystery', 'Drama,Thriller', 'Adventure,Comedy',
       'Drama,Sport', 'Horror,Mystery,Thriller', 'Adventure,Drama,Horror',
       'Comedy,Crime,Thriller', 'Drama,Fantasy,Music', 'Drama,Sci-Fi',
       'Dram

In [70]:
len(list_of_genres)

# we have 126 list of unique genre.

126

_Lets split those comma seperated genre values and create a new list which has just one value in it._

In [71]:
all_genres = list()

for genres in list_of_genres:
    genre = genres.split(',')
    for g in genre:
        all_genres.append(g)
        

all_genres = set(all_genres)
print(all_genres)

{'Family', 'Sport', 'Fantasy', 'Documentary', 'History', 'War', 'Horror', 'Music', 'Mystery', 'Musical', 'Animation', 'Action', 'Romance', 'Crime', 'Thriller', 'Biography', 'Sci-Fi', 'Comedy', 'Adventure', 'Western', 'Drama'}


In [72]:
df['genre_list'] = df['genres'].apply(lambda x: x.split(','))

_In order to seperate Genres we need to add each genre as a column to each row_

In [73]:
for genre in all_genres:
    df[genre] = np.zeros(shape=df.shape[0])

df.head()

,title,runtime,genres,rating,director,producer,cinematographer,writer,budget,domestic,...,Action,Romance,Crime,Thriller,Biography,Sci-Fi,Comedy,Adventure,Western,Drama
1,Lincoln,150.0,"Biography,Drama,History",7.4,Steven Spielberg,Kathleen Kennedy,Janusz Kaminski,Tony Kushner,65000000,182207973,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Season of the Witch,95.0,"Action,Adventure,Fantasy",5.4,Dominic Sena,Charles Roven,Amir Mokri,Bragi F. Schut,40000000,24827228,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Dallas Buyers Club,117.0,"Biography,Drama",8.0,Jean-Marc Vallée,Rachel Winter,Yves Bélanger,Craig Borten,5000000,27298285,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Interstellar,169.0,"Adventure,Drama,Sci-Fi",8.6,Christopher Nolan,Emma Thomas,Hoyte Van Hoytema,Jonathan Nolan,165000000,188017894,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Flipped,90.0,"Comedy,Drama,Romance",7.7,Rob Reiner,Alan Greisman,Thomas Del Ruth,Andrew Scheinman,14000000,1755212,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


_Now lets set 1 if the movie if of that genre else 0 to each column we just added_

In [74]:
for i, r in df.iterrows():
    if r['genre_list']:
        for g in r['genre_list']:
            df.loc[i,g] = 1

df.head(5)

,title,runtime,genres,rating,director,producer,cinematographer,writer,budget,domestic,...,Action,Romance,Crime,Thriller,Biography,Sci-Fi,Comedy,Adventure,Western,Drama
1,Lincoln,150.0,"Biography,Drama,History",7.4,Steven Spielberg,Kathleen Kennedy,Janusz Kaminski,Tony Kushner,65000000,182207973,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,Season of the Witch,95.0,"Action,Adventure,Fantasy",5.4,Dominic Sena,Charles Roven,Amir Mokri,Bragi F. Schut,40000000,24827228,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Dallas Buyers Club,117.0,"Biography,Drama",8.0,Jean-Marc Vallée,Rachel Winter,Yves Bélanger,Craig Borten,5000000,27298285,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,Interstellar,169.0,"Adventure,Drama,Sci-Fi",8.6,Christopher Nolan,Emma Thomas,Hoyte Van Hoytema,Jonathan Nolan,165000000,188017894,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
5,Flipped,90.0,"Comedy,Drama,Romance",7.7,Rob Reiner,Alan Greisman,Thomas Del Ruth,Andrew Scheinman,14000000,1755212,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [75]:
all_genres = list(all_genres)

df[all_genres] = df[all_genres].astype('int')

_We dont need Genre column now we can get rid of it._

In [76]:
df.head()

,title,runtime,genres,rating,director,producer,cinematographer,writer,budget,domestic,...,Action,Romance,Crime,Thriller,Biography,Sci-Fi,Comedy,Adventure,Western,Drama
1,Lincoln,150.0,"Biography,Drama,History",7.4,Steven Spielberg,Kathleen Kennedy,Janusz Kaminski,Tony Kushner,65000000,182207973,...,0,0,0,0,1,0,0,0,0,1
2,Season of the Witch,95.0,"Action,Adventure,Fantasy",5.4,Dominic Sena,Charles Roven,Amir Mokri,Bragi F. Schut,40000000,24827228,...,1,0,0,0,0,0,0,1,0,0
3,Dallas Buyers Club,117.0,"Biography,Drama",8.0,Jean-Marc Vallée,Rachel Winter,Yves Bélanger,Craig Borten,5000000,27298285,...,0,0,0,0,1,0,0,0,0,1
4,Interstellar,169.0,"Adventure,Drama,Sci-Fi",8.6,Christopher Nolan,Emma Thomas,Hoyte Van Hoytema,Jonathan Nolan,165000000,188017894,...,0,0,0,0,0,1,0,1,0,1
5,Flipped,90.0,"Comedy,Drama,Romance",7.7,Rob Reiner,Alan Greisman,Thomas Del Ruth,Andrew Scheinman,14000000,1755212,...,0,1,0,0,0,0,1,0,0,1


_Time to save this final dataframe to csv_

In [77]:
df.to_csv('datasets/final_df.csv')